In [ ]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point,LineString,  mapping


In [ ]:
import json
import geojson
from spacetx_spatial_analysis import depth_coordinate

## Make a napari viewer...

In [ ]:
v = napari.Viewer()

## Load spot table from 1 dataset

In [ ]:
# load one spotxgene table
all_spots = pd.read_csv("/Users/brian/Downloads/spot_table.csv")
genes = all_spots["gene"].unique()

In [ ]:
subsample = 1 # change to >1 if number of points is too high
colorlist = ["magenta", "red","blue","cyan", "green", "orange","red"]


In [ ]:
cindex = 0
for ii, gene_to_plot in enumerate(["Gad1","Gad2","Slc17a7","Cux2","Calb1","Meis2","Foxp2", "Atp1a3" , "Vxn", "Pcp4" , "Fezf2" , "Rorb" , "Gm11549"]):

    if gene_to_plot in genes:
        if gene_to_plot == "Slc17a7":
            spots = v.add_points(all_spots[all_spots["gene"]==gene_to_plot][["x_um","y_um"]].values[::subsample,:], symbol='disc',
                                 name = gene_to_plot+"+",edge_color= [0,0,0,0], size = 1.5,
                                 face_color= colorlist[ cindex % len(colorlist)] , blending = "translucent", opacity = 0.8)
        else:
            spots = v.add_points(all_spots[all_spots["gene"]==gene_to_plot][["x_um","y_um"]].values[::subsample,:], symbol='disc',
                     name = gene_to_plot+"+",edge_color= [0,0,0,0], size = 2.,
                     face_color= colorlist[ cindex % len(colorlist)] , blending = "translucent", opacity = 0.7)
        cindex = cindex + 1

In [ ]:
v.layers

### annotate the layers in that viewer in a single `shapes` layer
#### Naming/order Convention:
0. outline of entire area
1. Layer 1
1. Layer 2/3
2. Layer 4
3. Layer 5
4. Layer 6
5. white matter
6. single line segment, pointing from pia to white matter 

#### annotate the layer locations in a list

In [ ]:
shapes_layer = v.layers[-1].data[:-1]
column_points = v.layers[-1].data[-1]

In [ ]:
if len(v.layers[-1].data) != 8:
    print("please annotate the 7 polygons and 1 line segment described above")
else:
    print("8 annotations")

In [ ]:
# go get all the plotted points and note if they're in the annotated polygons
is_in_layer = {ii:[Polygon(p).intersects(Point(a))  for a  in all_spots[["x_um","y_um"]].values] for ii,p in enumerate(shapes_layer)}

In [ ]:
all_spots["layer"] = "outside_VISp"
for k in is_in_layer.keys():
    all_spots.loc[is_in_layer[k],["layer"]] = annotation_names[k]


In [ ]:
all_spots["layer"].unique()

In [ ]:
# now the spots have depth coordinates, which will be useful for detected spots v. column depth
all_spots["depth_um"] = depth_coordinate(all_spots[["x_um","y_um"]].values, column_points)

In [ ]:
all_spots

In [ ]:
all_spots.to_csv("Allen_MERFISH_spots_with_anatomy.csv")

In [ ]:
annotation_names = ["VISp", "VISp_I", "VISp_II/III", "VISp_IV", "VISp_V","VISp_VI", "VISp_wm", "column_axis"]


In [ ]:
annotations = [mapping(Polygon(p))  for ii,p in enumerate(shapes_layer)]
annotations.append(mapping(LineString(column_points)))
for ii,ai in enumerate(annotations):
    ai["name"] = annotation_names[ii]

a = geojson.GeometryCollection(annotations)

with open("Allen_MERFISH_annotations_geo.json",'w') as w:
    geojson.dump(a,w)

In [ ]:
with open("/Users/brian/work/data/stardist_rois.json",'r') as r:
    shapejson = geojson.load(r)

In [ ]:
shapes_to_add = [0.1078*4.*np.array(geo["coordinates"][0][::5]) for geo in shapejson["geometries"]]
for f in shapes_to_add:
    f[:,0] = f[:,0]+1154
    f[:,1] = f[:,1]+4548  #6237

In [ ]:
import imageio

In [ ]:
egdapi = imageio.imread("/Users/brian/work/mFISH__002_DAPI_Overview-ds.tif")

In [ ]:
print(10706.668188763051*0.1078)
print(44241.16940370426*0.1078 - 2048*.1078)

In [ ]:
v.add_image(egdapi.T, scale = [4.0*0.1078, 4.0*0.1078], translate = [1154, 4548])

In [ ]:
v.add_shapes(shapes_to_add ,
        shape_type='polygon',
        edge_width=1,
        edge_color='coral',
        face_color='royalblue',
        name='shapesjhjh')

In [ ]:
shapes_to_add[33]

In [ ]:
1.03*63./60.

In [ ]:
40/(6660-5383)

In [ ]:
1888-1845

In [ ]:
30

In [ ]:
1817-1786

In [ ]:
77-56